In [1]:
import json
import boto3
import asyncio
import argparse
from boto3.session import Session

In [2]:
async def invoke_agent(question):
    boto_session = Session()
    region_name = boto3.Session().region_name


    #SSM을 활용해서 Bedrock AgenticCore ARN 가져오기
    ssm_client = boto_session.client('ssm', region_name=region_name)
    agent_arn_response = ssm_client.get_parameter(
        Name="/rag_server/runtime_iam/agent_arn"
    )
    agentcore_client = boto3.client('bedrock-agentcore', region_name=region_name)

    
    #Streaming으로 Bedrock AgenticCore Application 호출하기
    loop = asyncio.get_event_loop()
    response = await loop.run_in_executor(
        None,
        lambda: agentcore_client.invoke_agent_runtime(
            agentRuntimeArn=agent_arn_response["Parameter"]["Value"],
            qualifier="DEFAULT",
            payload=json.dumps({"input_data": question})
        )
    )

    #응답을 객체로 받은뒤 계속적으로 오는 streaming 결과 출력
    for line in response['response'].iter_lines():
        if line and line.startswith(b'data: '):
            data = json.loads(line[6:].decode('utf-8'))
            if data['type'] == 'stream':
                print(data['content'], end='')
    print()

In [4]:
await invoke_agent("SageMaker에 대해 알려주세요")


안녕하세요! Amazon SageMaker AI에 대해 자세히 설명해드리겠습니다.

Amazon SageMaker AI는 **완전 관리형 머신러닝 서비스**로, ML 모델의 구축부터 훈련, 배포까지 전체 머신러닝 워크플로를 지원합니다. 인프라 관리 없이 대규모 모델 훈련이 가능하고, 다양한 배포 옵션과 CI/CD 기능을 제공하는 것이 특징입니다.

## 주요 구성 요소들을 살펴보면:

### 🔧 **모델 구축 도구**
- **SageMaker AI Studio 노트북** (Code-OSS 기반)
- **JupyterLab, RStudio, Code Editor** 등 다양한 개발 환경
- **15개 이상의 내장 알고리즘**과 **750개 이상의 사전 빌드된 모델** 제공

### 🚀 **모델 훈련**
- **1개부터 수천 개의 GPU**까지 확장 가능
- **SageMaker AI 분산 훈련 라이브러리** 또는 DeepSpeed, Horovod, Megatron 같은 타사 라이브러리 지원
- 사용한 만큼만 비용 지불하는 구조

### 📊 **데이터 관련 서비스**
- **SageMaker AI Data Wrangler**: 데이터 준비 시간을 몇 주에서 몇 분으로 단축
- **데이터 레이블 지정 서비스**: 이미지, 텍스트, 비디오 데이터에 고품질 레이블 추가

### 🤖 **자동화 도구들**
- **SageMaker AI Autopilot**: 테이블 형식 데이터로 최적의 ML 모델을 자동 구축
- **SageMaker AI JumpStart**: 150개 이상의 오픈소스 모델을 원클릭으로 배포
- **SageMaker AI Pipelines**: ML 전용 CI/CD 서비스로 end-to-end 워크플로 관리

### 🔍 **모델 관리 및 모니터링**
- **SageMaker AI Clarify**: 모델 편향 감지 및 설명 가능한 AI 기능
- **SageMaker AI Edge**: 엣지 디바이스에서의 모델 최적화 및 배포

### 🆓 **학습용 무료 